In [285]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib_venn as venn2
import piezo
import re
import gnomonicus
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')


In [6]:
WHO = pd.read_csv('/Users/kmalone/macbook_m1_backup/github/cryptic-catalogue/data/NC_000962.3_WHO-UCN-GTB-PCI-2021.7_v1.0_GARC1_RUS.csv')
training = pd.read_csv('/Users/kmalone/macbook_m1_backup/github/cryptic-catalogue/data/training/results/training_catalogue_FRS1.0.csv')

Considerations for comparing WHOv1 to training catalogue:  
1. WHOv1 lists upstream mutations with seemingly no bounds on distance from gene start site, while gnomonicus restricts to >= 100bp upstream. As these variants could fall into the boundaries of genes of interest, we need to translate upstream positions into genic ones (if they occur) e.g. geneX@c-500t could be geneY@c30t and geneY could be in our prediction list. 
To handle this, make translation table for WHOv1 upstream variants and see if any of  prediction genes are flagged.
2. WHOv1 also lists: synonymous mutations that have prediction == "U", strings containing 1+ mutations separated by "&", and a combination of both. Filtering out mutations whose string contains "&" and prediction == "U" will remove these and retain wildcard entries for comparison. I will however investigate these in a similar manner to 1) before removal.

In order to 're-code' the upstream variants, I need to make an annotation table with intergenic regions listed and labelled. I'll do this using NC_000962.3 annotation from a gff3 file, filtering for "gene" annotations.

In [222]:
# Read in annotation gff
annotation = pd.read_csv('/Users/kmalone/macbook_m1_backup/github/cryptic-catalogue/data/NC_000962.3.gff3', sep = "\t", header = None, comment="#")

# Filter for gene annotations
filtered_annotation = annotation[annotation[2] == 'gene']

# Extract gene ID and name from INFO field
filtered_annotation['Gene_ID'] = filtered_annotation[8].str.extract(r'ID=gene-(.*?);', expand=False)
filtered_annotation['Gene_Name'] = filtered_annotation[8].str.extract(r'gene=(.*?);', expand=False)

# Select and rename cols
annotation_table = filtered_annotation[[3, 4, 6, 'Gene_ID', 'Gene_Name']]

annotation_table.columns = ['Start', 'End', 'Strand', 'Gene_ID', 'Gene_Name']

# Sort by START position
annotation_table = annotation_table.sort_values(by='Start').reset_index(drop=True)

# Replace missing Gene_Names with Gene_IDs
annotation_table['Gene_Name'] = annotation_table['Gene_Name'].fillna(annotation_table['Gene_ID'])


# Create a new DataFrame to store the result
new_rows = []

# Iterate over the rows of the original DataFrame
for index, row in annotation_table.iterrows():

    # Calculate the start and end positions of the upstream region
    # If genes overlap, don't output an intergenic region
    if row['Start'] < annotation_table.iloc[index-1]['End'] - 1:
        new_rows.append(row)
        continue
    upstream_end = row['Start'] - 1
    upstream_start = annotation_table.iloc[index-1]['End'] + 1
    if row['Gene_ID'] == "Rv0001":
        upstream_start = 0
        upstream_end = 1
    # If gene_name is NaN, use gene_id instead
    if pd.isna(row['Gene_Name']):
        gene_name = f'upstream_{row["Gene_ID"]}'
    else:
        gene_name = f'upstream_{row["Gene_Name"]}'

    upstream_row = pd.Series({
        'Start': upstream_start,
        'End': upstream_end,
        'Strand': row['Strand'],
        'Gene_ID': f'upstream_{row["Gene_ID"]}',
        'Gene_Name': gene_name
    })

    # Append the new row to new_rows
    new_rows.append(row)
    new_rows.append(upstream_row)


# Create the new DataFrame
annotation_intergenic_table = pd.DataFrame(new_rows)
# Sort the DataFrame by the 'Start' column
annotation_intergenic_table = annotation_intergenic_table.sort_values(by='Start')
print(annotation_intergenic_table.to_string(index=False))

# save to file
annotation_intergenic_table.to_csv('/Users/kmalone/macbook_m1_backup/github/cryptic-catalogue/data/annotation_table.csv', index=False)

  Start     End Strand            Gene_ID          Gene_Name
      1    1524      +             Rv0001               dnaA
   1525    2051      +    upstream_Rv0002      upstream_dnaN
   2052    3260      +             Rv0002               dnaN
   3261    3279      +    upstream_Rv0003      upstream_recF
   3280    4437      +             Rv0003               recF
   4434    4997      +             Rv0004             Rv0004
   4998    5239      +    upstream_Rv0005      upstream_gyrB
   5240    7267      +             Rv0005               gyrB
   7268    7301      +    upstream_Rv0006      upstream_gyrA
   7302    9818      +             Rv0006               gyrA
   9819    9913      +    upstream_Rv0007    upstream_Rv0007
   9914   10828      +             Rv0007             Rv0007
  10829   10886      +    upstream_Rvnt01      upstream_ileT
  10887   10960      +             Rvnt01               ileT
  10961   11111      +    upstream_Rvnt02      upstream_alaT
  11112   11184      +  

Now, let's make a translation table for the WHOv1 upstream mutations using the annotation data above

In [ ]:
# Flag the mutation entries in the WHO catalogue that contain "-" followed by a string of digits
bp_upstream = WHO['MUTATION'].str.extract(r'-(\d+)', expand=False).astype(float)

# Filter for the WHO mutation entries that are upstream variants
upstream_variants = WHO[bp_upstream.notna()]

original_list = []
new_list = []

for mutation in upstream_variants['MUTATION']:
    # Extract the gene name and position from the mutation string
    gene_name = mutation.split('@')[0]

    # WHOv1 has mutation strings of n length separated by "&". Process each separately to allow for partial matches between catalogues
    mutations = mutation.split('&')

    if len(mutations) == 1:
        original_list.append(mutation)
        match = re.search(r'-(\d+)', mutation)
        if match:
            position = int(match.group(1))
        # Split the string by '@' to separate the lhv and rhv
        start, rest = mutation.split('@')

        ins_del_str = None
        if "ins" not in mutation or "del" not in mutation:
            # Extract lhv
            lhv_match = re.search(r'(.+)-\d+', rest)
        
            # Extract the lhv
            if lhv_match:
                lhv = lhv_match.group(1)
            else:
                lhv = None
        
            rhv_match = re.search(r'-\d+(.+)', rest)
            if rhv_match:
                rhv = rhv_match.group(1)
            else:
                rhv = None

        if "ins" in mutation or "del" in mutation:
            print(f'Found an ins/del: {mutation}')
            # Extract lhv
            rhv_match = re.search(r'_(del|ins)_([a-zA-Z]+)', rest)
            # Extract the lhv
            if rhv_match:
                rhv = rhv_match.group(2)
                ins_del_str = rhv_match.group(1)
            else:
                rhv = None
            lhv = None
            
            
        # Check if the gene name is in the Gene_Name column of the DataFrame
        if gene_name in annotation_intergenic_table['Gene_Name'].values:
            # Subtract the position from the Start column and print the result
            start_position = annotation_intergenic_table.loc[annotation_intergenic_table['Gene_Name'] == gene_name, 'Start'].iloc[0]
            end_position = annotation_intergenic_table.loc[annotation_intergenic_table['Gene_Name'] == gene_name, 'End'].iloc[0]
    
            search_position = start_position - position
            #print(f'{mutation}, Gene: {gene_name}, mutation_upstream: {position}, start position of gene: {start_position}, search position: {search_position}')
    
            # Iterate over the rows of the DataFrame
            for index, row in annotation_intergenic_table.iterrows():
                # Check if the start_position falls within the range defined by the Start and End columns of the current row
                if int(row['Start']) <= int(search_position) <= int(row['End']):
                    # If "_" in gene name string then position is intergenic, "upstream" 
                    if "_" in row['Gene_Name']:
                        print(f"upstream found {row['Gene_Name']}")
                        new_gene = row['Gene_Name'].split("_")[1]
                        new_position = (row['End'] + 1) - search_position
                        if not ins_del_str:
                            new_mutation = f'{new_gene}@{lhv}-{new_position}{rhv}'
                        elif ins_del_str:
                            new_mutation = f'{new_gene}@-{new_position}_{ins_del_str}_{rhv}'
                    # If there is no "_" in gene name then the search position falls within gene boundaries
                    else:
                        new_position = search_position
                        if not ins_del_str:
                            new_mutation = f'{row["Gene_Name"]}@{lhv}{new_position}{rhv}'
                        elif ins_del_str:
                            new_mutation = f'{row["Gene_Name"]}@{new_position}_{ins_del_str}_{rhv}'
                    #print(f'{mutation}, new name: {new_mutation}, Gene name: {row["Gene_Name"]} search_position: {search_position}, gene_start: {start_position}, gene_end: {end_position}, overlap start: {row["Start"]}, overlap end {row["End"]}')
        new_list.append(new_mutation)

    #If more than one mutation in string, process separately to facilitate partial matching
    if len(mutations) > 1:   
        #print(f'Found a long one {mutation}')
        for mutation_part in mutations:
            original_list.append(mutation)
            
            match = re.search(r'-(\d+)', mutation_part)
            if match:
                position = int(match.group(1))
            # Split the string by '@' to separate the lhv and rhv
            start, rest = mutation_part.split('@')
            gene_name = mutation_part.split('@')[0]
            
            
            ins_del_str = None
            if "ins" not in mutation_part or "del" not in mutation_part:
                # Extract lhv
                lhv_match = re.search(r'(.+)-\d+', rest)
            
                # Extract the lhv
                if lhv_match:
                    lhv = lhv_match.group(1)
                else:
                    lhv = None
            
                rhv_match = re.search(r'-\d+(.+)', rest)
                if rhv_match:
                    rhv = rhv_match.group(1)
                else:
                    rhv = None

            if "ins" in mutation_part or "del" in mutation_part:
                print(f'Found an ins/del: {mutation}')
                # Extract lhv
                rhv_match = re.search(r'_(del|ins)_([a-zA-Z]+)', rest)
                # Extract the lhv
                if rhv_match:
                    rhv = rhv_match.group(2)
                    ins_del_str = rhv_match.group(1)
                else:
                    rhv = None
                lhv = None

            # Check if the gene name is in the Gene_Name column of the DataFrame
            if gene_name in annotation_intergenic_table['Gene_Name'].values:
                # Subtract the position from the Start column and print the result
                start_position = annotation_intergenic_table.loc[annotation_intergenic_table['Gene_Name'] == gene_name, 'Start'].iloc[0]
                end_position = annotation_intergenic_table.loc[annotation_intergenic_table['Gene_Name'] == gene_name, 'End'].iloc[0]
            
                search_position = start_position - position
                #print(f'{mutation}, Gene: {gene_name}, mutation_upstream: {position}, start position of gene: {start_position}, search position: {search_position}')
            
                # Iterate over the rows of the DataFrame
                for index, row in annotation_intergenic_table.iterrows():
                    # Check if the start_position falls within the range defined by the Start and End columns of the current row
                    if int(row['Start']) <= int(search_position) <= int(row['End']):
                        # If a match is found, print the corresponding Gene_Name
                        if "_" in row['Gene_Name']:
                            new_gene = row['Gene_Name'].split("_")[1]
                            new_position = (row['End'] + 1) - search_position
                            if not ins_del_str:
                                new_mutation = f'{new_gene}@{lhv}-{new_position}{rhv}'
                            elif ins_del_str:
                                new_mutation = f'{new_gene}@-{new_position}_{ins_del_str}_{rhv}'
                        else:
                            new_position = search_position
                            if not ins_del_str:
                                new_mutation = f'{row["Gene_Name"]}@{lhv}{new_position}{rhv}'
                            elif ins_del_str:
                                new_mutation = f'{row["Gene_Name"]}@{new_position}_{ins_del_str}_{rhv}'
                        #print(f'{mutation}, new name: {new_mutation}, Gene name: {row["Gene_Name"]} search_position: {search_position}, gene_start: {start_position}, gene_end: {end_position}, overlap start: {row["Start"]}, overlap end {row["End"]}')
            new_list.append(new_mutation)
     
WHOv1_translation_table = pd.DataFrame({'original':original_list, 'new':new_list})

# save results
WHOv1_translation_table.to_csv('/Users/kmalone/macbook_m1_backup/github/cryptic-catalogue/data/WHOv1_translation_table.csv', index=False)

The results are as follows: 
* There are 48 unique entries in the WHO catalogue that have upstream positions listed which results in 109 instances when strings with "&" are unnested.
* Using the new annotation table, 85 of these 109 instances change with respect to the gene listed in the mutation string e.g. inhA@t-770a becomes fabG1@t-8a.  
* Of these 85, 45 instances map within a total of 5 genes (fabG1 x 4, mcr3 x 22, rv0681 x 1, rv1907c x 8 and rv2042c x 10).  
* Here, we are only concerned with those that map to gene of interest fabG1:  
  

| original      | new           | note                               |
|---------------|---------------|------------------------------------|
| inhA@c-522g  | fabG1@c1673680g | ==fabG1@P81A. This is already listed in WHOv1 separately. |
| inhA@c-40t   | fabG1@c1674162t | ==fabG1@G241G. Synonymous.        |
| inhA@g-154a  | fabG1@g1674048a | 2 entries of this, is also a synonymous mutation L-L. |  
  

* Therefore, I am happy to exclude all instances of upstream variants when comparing WHOv1 to a custom catalogue. From here on, they will be filtered out and will not influence the summary stats reported (unless stated).  
* Thee code above should be suitable for generating this data for future updates to WHOv1 once MUTATION GARC1 grammar and file format does not change.


In [292]:
#Select WHOv1 entries that are not considered upstream variants here
WHO_filtered = WHO[~bp_upstream.notna()]

# Filter those multiple variant strings that contain "&" and prediction == "U"
mult_mask = WHO['MUTATION'].str.contains('&') & (WHO['PREDICTION'] == "U")

WHO_filtered = WHO_filtered[~mult_mask]

WHO_filtered.to_csv("/Users/kmalone/macbook_m1_backup/github/cryptic-catalogue/data/NC_000962.3_WHO-UCN-GTB-PCI-2021.7_v1.0_GARC1_RUS_no_upstream.csv", index=False)

Let's compare training mutations to the filtered WHOv1 catalogue using piezo so we can deal with wildcards and nested variants efficiently (DA suggestion).

In [339]:
# For results
compare_res = {
    "drug": [],
    "c1": [],
    "c2": [],
    "c2_variant": [],
    "c2_listed_predict": [],
    "c1_predict_c2_variant": [],
    "match": []
}


# Read in filtered_WHO and convert to catalogue
WHOcat = piezo.ResistanceCatalogue("/Users/kmalone/macbook_m1_backup/github/cryptic-catalogue/data/NC_000962.3_WHO-UCN-GTB-PCI-2021.7_v1.0_GARC1_RUS_no_upstream.csv")

# Read in three letter code conversion for drugs
drug_codes = json.load(open('/Users/kmalone/macbook_m1_backup/github/cryptic-catalogue/data/drug_codes.json'))

# Compare training mutations to filtered WHOv1
for this_drug_long in training.DRUG.unique():
    # get drug code
    this_drug = drug_codes[this_drug_long]
    #skip drug predictions if drug is not handled by filtered WHOv1
    if this_drug not in WHO.DRUG.unique():
        print(f'{this_drug},{this_drug_long} not in catalogue. Skipping...')
        continue
    #filter the training catalogue for drug of interest
    this_df = training[training.DRUG == this_drug_long]
    #for each mutation listed for the drug of interest:
    for this_var in this_df.MUTATION:
        this_c2_predict = this_df[this_df['MUTATION'] == this_var]['PREDICTION'].iloc[0]
    
        compare_res["drug"].append(this_drug)
        compare_res["c2_variant"].append(this_var)
        compare_res["c2_listed_predict"].append(this_c2_predict)
        compare_res["c1"] = "WHOv1_filtered"
        compare_res["c2"] = "training"
        try:
            # get prediction for the mutation using piezo predict and the filtered WHOv1 catalogue
            this_c2_predict = str(WHOcat.predict(this_var)[this_drug])
            compare_res["c1_predict_c2_variant"].append(this_c2_predict)


        except (ValueError, AssertionError, TypeError):
            #print(f'{this_var} is failing, appending to not shared...')
            this_c2_predict = None
            compare_res["c1_predict_c2_variant"].append(this_c2_predict)

        this_match = this_c2_predict == this_c1_predict
        compare_res["match"].append(this_match)


compare_df = pd.DataFrame(compare_res)
compare_df.to_csv("/Users/kmalone/macbook_m1_backup/github/cryptic-catalogue/data/compare_training_NC_000962.3_WHO-UCN-GTB-PCI-2021.7_v1.0_GARC1_RUS_no_upstream.csv")

CIP,ciprofloxacin not in catalogue. Skipping...
OFX,ofloxacin not in catalogue. Skipping...


In [340]:
# Create a dictionary to store contingency tables for each drug
contingency_tables = {}

# Loop through each unique drug
for drug in compare_df.drug.unique():
    # Filter compare_df for the current drug
    drug_df = compare_df[compare_df['drug'] == drug]

    # Create contingency table
    contingency_table = pd.crosstab(drug_df['c2_listed_predict'], drug_df['c1_predict_c2_variant'],dropna=False)

    # Rename indices
    contingency_table = contingency_table.rename_axis(index='training', columns='WHOv1_filtered')

    # Add contingency table to dictionary
    contingency_tables[drug] = contingency_table

# Print contingency tables
# for drug, table in contingency_tables.items():
#     print(f"Contingency table for {drug}:")
#     print(table)
#     print('\n')

from tabulate import tabulate

# Loop through each unique drug
for drug, table in contingency_tables.items():
    print(f"Contingency table for {drug}:")
    print(tabulate(table, tablefmt='grid', headers='keys'))
    print('\n')

Contingency table for AMI:
+------------+-----+-----+-------+
| training   |   S |   U |   nan |
+============+=====+=====+=======+
| R          |   0 |  12 |     0 |
+------------+-----+-----+-------+
| S          |   2 | 230 |     1 |
+------------+-----+-----+-------+
| U          |   0 |  10 |     8 |
+------------+-----+-----+-------+


Contingency table for CAP:
+------------+-----+-----+-------+
| training   |   R |   U |   nan |
+============+=====+=====+=======+
| R          |   2 |   1 |     0 |
+------------+-----+-----+-------+
| S          |   0 |  40 |     1 |
+------------+-----+-----+-------+
| U          |   0 |   5 |     8 |
+------------+-----+-----+-------+


Contingency table for DLM:
+------------+-----+-----+-------+
| training   |   R |   U |   nan |
+============+=====+=====+=======+
| R          |   0 |  12 |     0 |
+------------+-----+-----+-------+
| S          |   0 |  31 |     1 |
+------------+-----+-----+-------+
| U          |   1 |   3 |     4 |
+----